In [ ]:
# pip3 install googletrans==3.1.0a0

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from googletrans import Translator
from transformers import Trainer, TrainingArguments, T5TokenizerFast, T5ForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## **Prueba traductor**

In [2]:
trans = Translator()
print(trans.translate('Pues hazme los deberes, furcia barata!').text)

Well, do my homework for me, you cheap whore!


## **Dataset**

In [ ]:
# csv = pd.read_csv('2_detox_dataset_trans.csv')
csv = pd.read_csv('english20k_dataset.csv')
df = pd.DataFrame(csv)

train_dataframe, val_dataframe = train_test_split(df, train_size=0.75, random_state=42) # random_state=42 for reproducibility

train_toxic_texts = list(train_dataframe['toxic_sentence'])
train_neutral_texts = list(train_dataframe['neutral_sentence'])

val_toxic_texts = list(val_dataframe['toxic_sentence'])
val_neutral_texts = list(val_dataframe['neutral_sentence'])

## **Detoxificador**

In [ ]:
model_name = "google/flan-t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class DetoxDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        # item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # item['labels'] = torch.tensor(self.target_encodings['input_ids'][idx])
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.target_encodings['input_ids'][idx].clone().detach()
        return item

In [ ]:
toxic_encodings_train = tokenizer(train_toxic_texts, truncation=True, padding=True, return_tensors="pt")
neutral_encodings_train = tokenizer(train_neutral_texts, truncation=True, padding=True, return_tensors="pt")

toxic_encodings_val = tokenizer(val_toxic_texts, truncation=True, padding=True, return_tensors="pt")
neutral_encodings_val = tokenizer(val_neutral_texts, truncation=True, padding=True, return_tensors="pt")

train_dataset = DetoxDataset(toxic_encodings_train, neutral_encodings_train)
val_dataset = DetoxDataset(toxic_encodings_val, neutral_encodings_val)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no", # NO validar en cada epoch
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-4,
    num_train_epochs=5,
    report_to=["none"], # para no pedir login de 'wandb' y otros
    fp16=True, # acelerar entrenaminento 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("flan-t5-detoxificado")
tokenizer.save_pretrained("flan-t5-detoxificado")

## **PRUEBAS DETOX**

In [ ]:
def detoxify_sentence(text: str):
    # Tokenizar la oración tóxica
    inputs: dict = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Mover los tensores a la GPU, si está disponible
    inputs = {key: val.to(device) for key, val in inputs.items()}

    outputs = model.generate(inputs["input_ids"])
    texto_neutralizado = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return texto_neutralizado

In [ ]:
test_sentences = [
    "shut your angry ass up",
    "You are a stupid asshole."
]

for sentence in test_sentences:
    neutral_sentence = detoxify_sentence(sentence)
    print(f"Original: {sentence} -> Neutral: {neutral_sentence}") 

Original: shut your angry ass up -> Neutral: shut your angry ass up
Original: You are a stupid asshole. -> Neutral: You are stupid asshole.


---

---

## **BUCLE DE TRADUCCIÓN** (26 mins)

In [23]:
csv = pd.read_csv('1_detox_dataset.csv')
df = pd.DataFrame(csv)

trans = Translator()
# print(trans.translate(df.iloc[400]['toxic_sentence'], dest='en').text)
print(trans.translate("ታምራት ነገራ ፅንፈኞችን ከላይ እስከታች ሰብስበህ ሁለት ሰንበት ፀበልና ስልጠና ስጥልን", dest='en').text)

Tamrat Negara gathered the extremists from top to bottom and gave us two Sabbath prayers and training.


---

In [ ]:
df = pd.DataFrame(pd.read_csv("1_detox_dataset.csv"))
df_trans = pd.DataFrame(columns=["toxic_sentence", "neutral_sentence"])
traductor = Translator()

for index in range(len(df)):
    toxic_translated = traductor.translate(df.iloc[index]['toxic_sentence'], dest='en').text
    neutral_translated = traductor.translate(df.iloc[index]['neutral_sentence'], dest='en').text
    df_trans.loc[len(df_trans)] = [toxic_translated, neutral_translated]
    # break

df_trans.to_csv("2_detox_dataset_trans.csv", index=False)